<a href="https://colab.research.google.com/github/batucul/calisma/blob/main/Sikayetvar_grabber_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas openpyxl gradio pyngrok==4.1.1 lxml requests beautifulsoup4

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import lxml
import re
import time
import random

def setup_session():
    """Requests oturumunu yapılandırır ve döndürür."""
    session = requests.Session()
    retry = Retry(connect=5, backoff_factor=1)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def get_page_count(session, url):
    """Belirtilen URL için sayfa sayısını alır."""
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.28 Safari/537.36'}
    site_request = session.get(url, headers=headers).text
    soup = BeautifulSoup(site_request, "lxml")
    return int(soup.find_all("a", class_="page")[-1].text)

def get_complaint_links(session, url, page_count):
    """Tüm şikayet linklerini toplar ve döndürür."""
    yorum_linkleri = []
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.28 Safari/537.36'}
    for page in range(1, page_count + 1):
        page_url = f"{url}?page={page}"
        page_request = session.get(page_url, headers=headers).text
        soup = BeautifulSoup(page_request, "lxml")
        for container in soup.find_all("div", class_="read-more-container"):
            read_more_div = container.find("div", class_="read-more")
            if read_more_div and read_more_div.has_attr('data-url'):
                data_url = read_more_div['data-url']
                yorum_linkleri.append("https://www.sikayetvar.com" + data_url)
    return yorum_linkleri

def scrape_complaint(session, url):
    """Tek bir şikayet sayfasını kazır."""
    review_site_request = session.get(url, headers=headers).text
    soup = BeautifulSoup(review_site_request, "lxml")
    try:
        baslik = soup.find("h1", class_="complaint-detail-title").text.strip()
        yorum = soup.find("div", class_="complaint-detail-description").text.strip()
        tarih = soup.find("div", class_="post-time").get_text().split("  ")[0]
        goruntulenme = soup.find("span", class_="js-view-count").get_text()
        kullanici_adi = soup.find("span", class_="username").text.strip()
        return [url, kullanici_adi, tarih, baslik, yorum, goruntulenme]
    except Exception as e:
        print(f"Error processing {url}: {e}")
        return None

def main():
    kargo = ['tazedirekt']
    df_columns = ['PrimaryKey', 'href', 'kullanici_adi', 'tarih', 'baslik', 'yorum', 'goruntulenme']
    data = []

    session = setup_session()
    for firma in kargo:
        url = f"https://www.sikayetvar.com/{firma}"
        page_count = get_page_count(session, url)
        yorum_linkleri = get_complaint_links(session, url, page_count)

        for idx, link in enumerate(yorum_linkleri, 1):
            time.sleep(random.uniform(1, 2))
            complaint_data = scrape_complaint(session, link)
            if complaint_data:
                data.append([idx] + complaint_data)
                print(f"Scraping progress: {idx}/{len(yorum_linkleri)}")

    df = pd.DataFrame(data, columns=df_columns)
    excel_filename = "scraped_data.xlsx"
    df.to_excel(excel_filename, index=False)
    print("Scraping completed and data written to 'scraped_data.xlsx'")

if __name__ == "__main__":
    main()


In [ ]:
#%%Gradio APP
import pandas as pd
from bs4 import BeautifulSoup
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import lxml
import re
import time
import random
import gradio as gr
from tqdm import tqdm


df_columns = ['PrimaryKey', 'href', 'kullanici_adi', 'tarih', 'baslik', 'yorum', 'goruntulenme']


def setup_session():
    """Requests oturumunu yapılandırır ve döndürür."""
    session = requests.Session()
    retry = Retry(connect=5, backoff_factor=1)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def get_page_count(session, url):
    """Belirtilen URL için sayfa sayısını alır."""
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.28 Safari/537.36'}
    site_request = session.get(url, headers=headers).text
    soup = BeautifulSoup(site_request, "lxml")
    return int(soup.find_all("a", class_="page")[-1].text)

def get_complaint_links(session, url, page_count):
    """Tüm şikayet linklerini toplar ve döndürür."""
    yorum_linkleri = []
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.28 Safari/537.36'}
    for page in range(1, page_count + 1):
        page_url = f"{url}?page={page}"
        page_request = session.get(page_url, headers=headers).text
        soup = BeautifulSoup(page_request, "lxml")
        for container in soup.find_all("div", class_="read-more-container"):
            read_more_div = container.find("div", class_="read-more")
            if read_more_div and read_more_div.has_attr('data-url'):
                data_url = read_more_div['data-url']
                yorum_linkleri.append("https://www.sikayetvar.com" + data_url)
    return yorum_linkleri

def scrape_complaint(session, url):
    """Tek bir şikayet sayfasını kazır."""
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.28 Safari/537.36'}
    review_site_request = session.get(url, headers=headers).text
    soup = BeautifulSoup(review_site_request, "lxml")
    try:
        baslik = soup.find("h1", class_="complaint-detail-title").text.strip()
        yorum = soup.find("div", class_="complaint-detail-description").text.strip()
        tarih = soup.find("div", class_="post-time").get_text().split("  ")[0]
        goruntulenme = soup.find("span", class_="js-view-count").get_text()
        kullanici_adi = soup.find("span", class_="username").text.strip()
        return [url, kullanici_adi, tarih, baslik, yorum, goruntulenme]
    except Exception as e:
        print(f"Error processing {url}: {e}")
        return None

def scrape_and_save(kargo,  progress=gr.Progress()):
    session = setup_session()
    url = f"https://www.sikayetvar.com/{kargo}"
    page_count = get_page_count(session, url)
    yorum_linkleri = get_complaint_links(session, url, page_count)

    data = []
    for idx, link in enumerate(yorum_linkleri):
        time.sleep(random.uniform(0, 1))
        complaint_data = scrape_complaint(session, link)
        if complaint_data:
            data.append([idx] + complaint_data)
            print(f"Scraping progress: {idx}/{len(yorum_linkleri)}")
        progress((idx + 1) / len(yorum_linkleri))

    df = pd.DataFrame(data, columns=df_columns)
    excel_filename = f"{kargo}_scraped_data.xlsx"
    df.to_excel(excel_filename, index=False)
    return excel_filename
# Create a Gradio interface
iface = gr.Interface(
    fn=scrape_and_save,
    inputs=gr.Textbox(label="Firma"),
    outputs=gr.File(),
    title="Şikayetvar Verilerini Çekme",
    description= """1- Şikayetlerini indirmek istediğiniz firmayı şikayetvar.com'da bulun
2- Firmayı bulduktan sonra adres çubuğundaki firma ismini aşağıya girin
3- Örneğin: https://www.sikayetvar.com/aras-kargo ise programa aras-kargo yazın"""
).queue().launch(share=True, debug=True, show_error=True)